<a href="https://colab.research.google.com/github/dsynderg/CS-180-assignments/blob/main/APINotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Create a Google AI Studio Account**
Go to aistudio.google.com <br>
Sign in with Google account <br>
Get your free API key and enter it in the code below

In [ ]:
# Install the Gemini SDK
!pip install -q google-generativeai

import google.generativeai as genai

# Configure the API with your key
genai.configure(api_key="<YOUR API KEY>")

model = genai.GenerativeModel('gemini-2.5-flash')

print("AI Assistant Ready!")

With your model set up you can now call it to generate content:

In [ ]:
# Send a prompt to the AI
response = model.generate_content("What is Python programming?")

# Print what the AI said back
print(response.text)

We can also set it up to get a question from the user:

In [ ]:
user_question = input("Ask me anything: ")

# Send to AI
response = model.generate_content(user_question)

# Show the answer
print("\nAI's Answer:")
print(response.text)

Giving the system an instruction will get it to answer the question is a specific way

In [ ]:
model = genai.GenerativeModel('gemini-2.5-flash', system_instruction ="You are a cat. Your name is Neko.")

response = model.generate_content(contents = "Hello there")

# Show the answer
print("\nAI's Answer:")
print(response.text)

Gemini can take in more than just text input. Below we can try passing in an image to the generate content function.

In [ ]:
from PIL import Image

model = genai.GenerativeModel('gemini-2.5-flash')

#Download an image from the web. Open files by clicking the file icon on the left. Drag your image into sample_data and copy the path to the file"
image = Image.open("<PATH TO YOUR IMAGE>")

response = model.generate_content(contents=[image, "tell me about this image"])
print(response.text)

Previous questions are not stored when we call generate content. See below how the model no longer knows my name despite being told it earlier:

In [ ]:
response = model.generate_content("My name is Emily. What is your name?")
print(response.text)

response = model.generate_content("What is my name?")
print(response.text)

As a solution, we can create a chat and use it to remember the context of our conversation. Below the hood, Gemini still calls generate_content, but will also pass in the previous context of our conversation.

In [ ]:
chat = model.start_chat()

response = chat.send_message("My name is Emily. What is your name?")
print(response.text)
response = chat.send_message("What is my name?")
print(response.text)

Now that we know a bit about generating content using the gemini API, let's make a choose your own adventure game! Some skeleton code is provided below. Write the prompts so that your choose your own adventure AI will work correctly.

In [ ]:
from IPython.display import display, Markdown, clear_output
import time


class AdventureGame:
    def __init__(self, theme="fantasy"):
        self.theme = theme

        system_prompt = f"""<ENTER A PROMPT HERE EXPLAINING WHAT THE MODEL SHOULD GENERALLY DO>"""

        self.model = genai.GenerativeModel(
            'gemini-2.5-flash',
            system_instruction=system_prompt
        )

        self.chat = self.model.start_chat()


    def start_game(self):
        prompt = f"""<ENTER A PROMPT HERE TO START THE GAME.>"""

        response = self.chat.send_message(prompt)
        scene = response.text
        return scene

    def continue_story(self, choice_number):
        prompt = f"<ENTER A PROMPT HERE TO PASS THE CHOICE NUMBER THE TO THE MODEL>"

        response = self.chat.send_message(prompt)
        scene = response.text
        return scene

    def end_game(self):
        prompt = """<ENTER A PROMPT HERE TO CREATE A SATISFYING CONCLUSION THE GAME>"""

        response = self.chat.send_message(prompt)
        return response.text

In [ ]:
def display_scene(text):
    """Pretty display for scenes"""
    display(Markdown(f"---\n\n{text}\n\n---"))

def get_player_choice():
    """Get valid player input"""
    while True:
        try:
            choice = input("\nEnter your choice (1-3): ").strip().lower()
            choice_num = int(choice)
            if 1 <= choice_num <= 3:
                return choice_num
            print("Please enter 1, 2, or 3")
        except ValueError:
            print("Please enter a valid number")

In [ ]:
theme_choice = input("What will the theme of your story be?")

# Initialize game
game = AdventureGame(theme=theme_choice)
print(f"\n🎮 Starting your {theme_choice} adventure...\n")
time.sleep(1)

# Opening scene
scene = game.start_game()
display_scene(scene)

# Game loop
turn_count = 0
max_turns = 3

while turn_count < max_turns:
    choice = get_player_choice()

    print("\n⏳ Generating next scene...")
    scene = game.continue_story(choice)
    display_scene(scene)
    turn_count += 1

    if (turn_count == max_turns - 1):
      game.end_game()
      break

print(f"\n🎉 Game Over! You played {turn_count} turns.")
print(f"🎲 Play again with a different theme for a new experience!")

Once you have a working adventure game, let's work on adding extra functionality. Some ideas:<br>
1. Have the adventure game theme be based on a passed in image
2. add custom player actions
3. Add character stats that affect the story